# HDB Resale Price Predictor & Visualisation

This project aims to create a data pipeline with the help of availale APIs (Data.gov.sg and OneMap) to build a web-based application for
1. HDB Price visualisation
2. HDB Price prediction

The prototype aims to read latest data directly from data.gov.sg and perform ETL (Extract, Transform, and Load) to a local/web database of choice.

In [1]:
import requests
from requests.exceptions import HTTPError
import pandas as pd
from pprint import pprint
import json

In [6]:
def get_token(location):
    '''
    Function to check if API token is still valid and updates API token if outdated
    Returns the API token
    '''
    try:
        with open(location, 'r+') as fp:
            file = fp.read()
            data = json.loads(file)
            response = requests.post("https://developers.onemap.sg/privateapi/auth/post/getToken", data=data)
            token = response.json()
            if token['access_token'] != data['access_token']:
                print(f"New token found")
                data['access_token'] = token['access_token']
                data['expiry_timestamp'] = token['expiry_timestamp']
                fp.seek(0)
                json.dump(data, fp = fp, indent=4)
                print('Updated token json')
                data = json.loads(file)
            return data['access_token']
    except Exception as err:
        print(err)

credentials = get_token("venv/onemap.json")

def datagovsg_api_call(url, sort = 'month desc', limit = 100, months=[1,2,3,4,5,6,7,8,9,10,11,12], years=["2023"]):
    '''
    Function to build the API call and construct the pandas dataframe
    Inputs:
        url: url for API, with resource_id parameters
        sort: field, by ascending/desc
        limit: maximum entries (API default by OneMap is 100, if not specified)
        years: list of years data required
    Returns a pandas dataframe of the data
    '''
    month_dict = '{"month":['
    for year in years:
        for month in months: # months 1-12
            month_dict = month_dict + f'"{year}-{str(month).zfill(2)}", '
    month_dict = month_dict[:-2] # Cancel out extra strings <, >
    month_dict = month_dict + ']}'
    url = url+f'&sort={sort}&filters={month_dict}'
    if limit: # API call's default is 100 even without specifying
        print(f'Call limit : {limit}')
        url = url+f'&limit={limit}'
    pprint(f'API call = {url}')
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        df = pd.DataFrame(data['result']['records'])
    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')
    except Exception as err:
        print(f'Other error occurred: {err}')
    else:
        return df

In [14]:
df = datagovsg_api_call('https://data.gov.sg/api/action/datastore_search?resource_id=f1765b54-a209-4718-8d38-a39237f502b3', months = [4],limit = 10000)
df

Call limit : 10000
('API call = '
 'https://data.gov.sg/api/action/datastore_search?resource_id=f1765b54-a209-4718-8d38-a39237f502b3&sort=month '
 'desc&filters={"month":["2023-04"]}&limit=10000')


,town,flat_type,flat_model,floor_area_sqm,street_name,resale_price,month,remaining_lease,lease_commence_date,storey_range,_id,block
0,ANG MO KIO,2 ROOM,Improved,44,ANG MO KIO AVE 3,298000,2023-04,53 years 09 months,1978,07 TO 09,143402,314
1,ANG MO KIO,2 ROOM,Improved,44,ANG MO KIO AVE 8,305000,2023-04,56 years 02 months,1980,07 TO 09,143405,510
2,ANG MO KIO,3 ROOM,New Generation,82,ANG MO KIO AVE 1,372000,2023-04,53 years 04 months,1977,01 TO 03,143421,220
3,ANG MO KIO,3 ROOM,Model A,70,ANG MO KIO AVE 1,585000,2023-04,88 years 05 months,2012,19 TO 21,143422,308B
4,ANG MO KIO,3 ROOM,New Generation,68,ANG MO KIO AVE 10,388000,2023-04,56 years 10 months,1981,01 TO 03,143464,542
...,...,...,...,...,...,...,...,...,...,...,...,...
2186,YISHUN,EXECUTIVE,Maisonette,154,YISHUN ST 22,808000,2023-04,61 years 02 months,1985,01 TO 03,152791,258
2187,YISHUN,EXECUTIVE,Apartment,169,YISHUN ST 22,892000,2023-04,68 years 04 months,1992,01 TO 03,152792,293
2188,YISHUN,EXECUTIVE,Apartment,164,YISHUN ST 61,935000,2023-04,68 years 02 months,1992,04 TO 06,152795,604
2189,YISHUN,EXECUTIVE,Apartment,142,YISHUN ST 71,780000,2023-04,62 years 02 months,1986,10 TO 12,152797,723


In [15]:
# from dataprep.eda import create_report
# create_report(df).show()

## Data wrangling and feature engineering steps

1. Reindexed dataframe using _id (unique to every resale transaction)
2. Categorised towns into regions (North, West, East, North-East, Central) based on HDB's categorisation https://www.hdb.gov.sg/about-us/history/hdb-towns-your-home
3. Changed room types into float values, with Executive as 4.5 rooms (extra study/balcony), and Multigeneration 6 rooms
4. Floor area converted to float values
5. Month was converted into datetime format, to be used to detrend the time series moving average
6. Year/Month was separated into Year and Month for visualisation purposes
7. Storey range was converted to avg_storey, since unable to determine the floor, the avg floor would be used (every value is a difference of 3 storeys)
8. Remaining lease was converted into remaining months (float)

Lastly, location plays a huge role in house pricing, hence
1. Using street name and block, I utilized OneMap API to obtain the latitude, longitude, and postal codes of each flat https://www.onemap.gov.sg/docs
2. The location of all MRT stations was also obtained using OneMap API and saved as a json file locally
3. Using the two data above, I am able to determine the nearest MRT station
4. The minimum travelling time (walk and public transport) to the nearest MRT will be an additional feature of the dataset

In [12]:
def clean_df(df):
    '''
    function to clean the raw dataframe
    '''
    from time import sleep
    # Start
    # set index to overall id
    df.set_index('_id', inplace=True)
        
    # Create feature "rooms", "avg_storey"
    def categorise_rooms(flat_type):
        '''
        Helper function for categorising number of rooms
        '''
        if flat_type[0] == 'E':
            return 4.5
        elif flat_type[0] == 'M':
            return 6.0
        else:
            return float(flat_type[0])
        
    df['rooms'] = df['flat_type'].apply(categorise_rooms)
    df['avg_storey'] = df['storey_range'].apply(lambda x: (int(x[:2])+int(x[-2:]))/2)

    # Change dtypes
    df['lease_commence_date'] = df['lease_commence_date'].astype('int')
    df['resale_price'] = df['resale_price'].astype('float')
    df['floor_area_sqm'] = df['floor_area_sqm'].astype('float')
    df['timeseries_month'] = pd.to_datetime(df['month'], format="%Y-%m-%d")
    df['year'] = df['timeseries_month'].dt.year
    df['month'] = df['timeseries_month'].dt.month
    
    # Calculate remaining_lease
    def year_month_to_year(remaining_lease):
        '''
        Helper function to change year & months, into years (float)
        '''
        remaining_lease = remaining_lease.split(' ')
        if len(remaining_lease) > 2:
            year = float(remaining_lease[0]) + float(remaining_lease[2])/12
        else:
            year = float(remaining_lease[0])
        return year
    
    df['remaining_lease'] = df['remaining_lease'].apply(year_month_to_year)

    # Change capitalization of strings
    for column in df.columns:
        if df[column].dtype == 'O':
            df[column] = df[column].str.title()
    
    # Update address abbreviations for onemap API call
    df['original_street_name'] = df['street_name']
    abbreviations = {'Sth':'South', 
                     '[S][t][^.ri]':'Street ', 
                     '[S][t]$':'Street',
                     '[S][t][.]':'Saint', 
                     'Nth':'North', 
                     'Ave':'Avenue', 
                     'Dr':'Drive', 
                     'Rd':'Road'}
    for abbreviation, full in abbreviations.items():
        df['street_name'] = df['street_name'].str.replace(abbreviation, full, regex=True)
    
    # Categorise town regions
    town_regions = {'Sembawang' : 'North',
                'Woodlands' : 'North',
                'Yishun' : 'North',
                'Ang Mo Kio' : 'North-East',
                'Hougang' : 'North-East',
                'Punggol' : 'North-East',
                'Sengkang' : 'North-East',
                'Serangoon' : 'North-East',
                'Bedok' : 'East',
                'Pasir Ris' : 'East',
                'Tampines' : 'East',
                'Bukit Batok' : 'West',
                'Bukit Panjang' : 'West',
                'Choa Chu Kang' : 'West',
                'Clementi' : 'West',
                'Jurong East' : 'West',
                'Jurong West' : 'West',
                'Tengah' : 'West',
                'Bishan' : 'Central',
                'Bukit Merah' : 'Central',
                'Bukit Timah' : 'Central',
                'Central Area' : 'Central',
                'Geylang' : 'Central',
                'Kallang/Whampoa' : 'Central',
                'Marine Parade' : 'Central',
                'Queenstown' : 'Central',
                'Toa Payoh' : 'Central'}      
    df['region'] = df['town'].apply(lambda x: town_regions[x])

    # Getting latitude, longitude, postal code
    def get_lat_long(df):
        '''
        API call to get latitude, longitude, and postal code
        Incorporates sleep time to not exceed a max of 250 calls per min
        '''
        sleep(0.15)
        address = df['block'] + ', ' + df['street_name']
        try:
            call = f'https://developers.onemap.sg/commonapi/search?searchVal={address}&returnGeom=Y&getAddrDetails=Y'
            response = requests.get(call)
            response.raise_for_status()
            data = response.json()
            return data['results'][0]['LATITUDE'] + ',' + data['results'][0]['LONGITUDE'] + ' ' + data['results'][0]['POSTAL']
        except HTTPError as http_err:
            print(f'HTTP error occurred during get_lat_long: {http_err}')
        except Exception as err:
            print(f'Error occurred during get_lat_long: {err} on the following call:')
            pprint(call)
            return '0,0 0' # Still return 0 values

    df['position'] = df.apply(get_lat_long, axis=1)
    try:
        df['postal_code'] = df['position'].apply(lambda x: x.split()[1]).astype('int')
        df['lat_long'] = df['position'].apply(lambda x: x.split()[0])
        # I need another split here to get floats
        df['lat'] = df['lat_long'].apply(lambda x: float(x.split(',')[0]))
        df['long'] = df['lat_long'].apply(lambda x: float(x.split(',')[1]))
        
    except Exception as err:
        print(f'Error splitting postal_code from lat_long: {err}')
    else:
        # Reorder columns
        df = df[['resale_price', 'year', 'month', 'timeseries_month', 'region', 'town', 'rooms', 'avg_storey', 'floor_area_sqm', 'remaining_lease',
                'lat_long', 'lat', 'long', 'postal_code']]
                # Unused columns - 'block', 'street_name', 'original_street_name', 'lease_commence_date', 'flat_model', 'storey_range', 'flat_type'
    return df

In [15]:
df = clean_df(df)
df.dtypes

Error occurred during get_lat_long: list index out of range on the following call:
('https://developers.onemap.sg/commonapi/search?searchVal=166, Street rling '
 'Road&returnGeom=Y&getAddrDetails=Y')
Error occurred during get_lat_long: list index out of range on the following call:
('https://developers.onemap.sg/commonapi/search?searchVal=171, Street rling '
 'Road&returnGeom=Y&getAddrDetails=Y')
Error occurred during get_lat_long: list index out of range on the following call:
('https://developers.onemap.sg/commonapi/search?searchVal=52, Street athmore '
 'Avenue&returnGeom=Y&getAddrDetails=Y')
Error occurred during get_lat_long: list index out of range on the following call:
('https://developers.onemap.sg/commonapi/search?searchVal=49, Street athmore '
 'Avenue&returnGeom=Y&getAddrDetails=Y')
Error occurred during get_lat_long: list index out of range on the following call:
('https://developers.onemap.sg/commonapi/search?searchVal=51, Street athmore '
 'Avenue&returnGeom=Y&getAddrDet

town                            object
flat_type                       object
flat_model                      object
floor_area_sqm                 float64
street_name                     object
resale_price                   float64
month                            int64
remaining_lease                float64
lease_commence_date              int32
storey_range                    object
block                           object
rooms                          float64
avg_storey                     float64
timeseries_month        datetime64[ns]
year                             int64
original_street_name            object
region                          object
position                        object
dtype: object

In [21]:
df

,town,flat_type,flat_model,floor_area_sqm,street_name,resale_price,month,remaining_lease,lease_commence_date,storey_range,block,rooms,avg_storey,timeseries_month,year,original_street_name,region,position
_id,,,,,,,,,,,,,,,,,,
143402,Ang Mo Kio,2 Room,Improved,44.0,Ang Mo Kio Avenue 3,298000.0,4,53.750000,1978,07 To 09,314,2.0,8.0,2023-04-01,2023,Ang Mo Kio Ave 3,North-East,"1.36622707120636,103.850085858983 560314"
143405,Ang Mo Kio,2 Room,Improved,44.0,Ang Mo Kio Avenue 8,305000.0,4,56.166667,1980,07 To 09,510,2.0,8.0,2023-04-01,2023,Ang Mo Kio Ave 8,North-East,"1.37340092645025,103.849073244454 560510"
143421,Ang Mo Kio,3 Room,New Generation,82.0,Ang Mo Kio Avenue 1,372000.0,4,53.333333,1977,01 To 03,220,3.0,2.0,2023-04-01,2023,Ang Mo Kio Ave 1,North-East,"1.36558833593063,103.840518883254 560220"
143422,Ang Mo Kio,3 Room,Model A,70.0,Ang Mo Kio Avenue 1,585000.0,4,88.416667,2012,19 To 21,308B,3.0,20.0,2023-04-01,2023,Ang Mo Kio Ave 1,North-East,"1.3652661423815,103.844538059044 562308"
143464,Ang Mo Kio,3 Room,New Generation,68.0,Ang Mo Kio Avenue 10,388000.0,4,56.833333,1981,01 To 03,542,3.0,2.0,2023-04-01,2023,Ang Mo Kio Ave 10,North-East,"1.37473841331656,103.855876768354 560542"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152791,Yishun,Executive,Maisonette,154.0,Yishun Street 22,808000.0,4,61.166667,1985,01 To 03,258,4.5,2.0,2023-04-01,2023,Yishun St 22,North,"1.43515573230958,103.839804271982 760258"
152792,Yishun,Executive,Apartment,169.0,Yishun Street 22,892000.0,4,68.333333,1992,01 To 03,293,4.5,2.0,2023-04-01,2023,Yishun St 22,North,"1.43611425409216,103.837605235958 760293"
152795,Yishun,Executive,Apartment,164.0,Yishun Street 61,935000.0,4,68.166667,1992,04 To 06,604,4.5,5.0,2023-04-01,2023,Yishun St 61,North,"1.42171348124463,103.83554529409 760604"


In [22]:
save = input('Save? y/n')
if save == 'y':
    df.to_csv('static/2023_APR.csv')

In [2]:
def update_mrt_coordinates(mrt_stations=None):
    '''
    Function to API call for MRT station coordinates
    Input: list of mrt station names, default to All stations if nothing is given
    '''
    if not mrt_stations:
        mrt_stations = ['Admiralty MRT', 'Aljunied MRT', 'Ang Mo Kio MRT', 'Bakau LRT', 'Bangkit LRT', 'Bartley MRT', 'Bayfront MRT',
                        'Bayshore MRT', 'Beauty World MRT', 'Bedok MRT', 'Bedok North MRT', 'Bedok Reservoir MRT', 'Bencoolen MRT',
                        'Bendemeer MRT', 'Bishan MRT', 'Boon Keng MRT', 'Boon Lay MRT', 'Botanic Gardens MRT', 'Braddell MRT',
                        'Bras Basah MRT', 'Buangkok MRT', 'Bugis MRT', 'Bukit Batok MRT', 'Bukit Brown MRT', 'Bukit Gombak MRT',
                        'Bukit Panjang MRT', 'Buona Vista MRT', 'Caldecott MRT', 'Cashew MRT', 'Changi Airport MRT',
                        'Chinatown MRT', 'Chinese Garden MRT', 'Choa Chu Kang MRT', 'City Hall MRT', 'Clarke Quay MRT',
                        'Clementi MRT', 'Commonwealth MRT', 'Compassvale LRT', 'Cove LRT', 'Dakota MRT', 'Dhoby Ghaut MRT',
                        'Downtown MRT', 'Xilin MRT', 'Tampines East MRT', 'Mayflower MRT', 'Upper Thomson MRT',
                        'Lentor MRT', 'Woodlands North MRT', 'Woodlands South MRT', 'Esplanade MRT', 'Eunos MRT',
                        'Expo MRT', 'Fajar LRT', 'Farmway LRT', 'Farrer Park MRT', 'Fort Canning MRT',
                        'Gardens by the Bay MRT', 'Geylang Bahru MRT', 'HarbourFront MRT', 'Haw Par Villa MRT', 'Hillview MRT',
                        'Holland Village MRT', 'Hougang MRT', 'Jalan Besar MRT', 'Joo Koon MRT', 'Jurong East MRT',
                        'Jurong West MRT', 'Kadaloor LRT', 'Kaki Bukit MRT', 'Kallang MRT', 'Kembangan MRT', 'Keppel MRT',
                        'King Albert Park MRT', 'Kovan MRT', 'Kranji MRT', 'Labrador Park MRT', 'Lakeside MRT', 'Lavender MRT',
                        'Layar LRT', 'Little India MRT', 'Lorong Chuan MRT', 'MacPherson MRT', 'Marina Bay MRT', 'Marina South Pier MRT',
                        'Marsiling MRT', 'Marymount MRT', 'Mattar MRT', 'Meridian LRT', 'Mountbatten MRT',
                        'Newton MRT', 'Nibong LRT', 'Nicoll Highway MRT', 'Novena MRT', 'Oasis LRT', 'One-North MRT', 'Orchard MRT',
                        'Outram Park MRT', 'Paya Lebar MRT', 'Pasir Ris MRT', 'Paya Lebar MRT', 'Pasir Ris MRT', 'Paya Lebar MRT', 'Pasir Ris MRT', 
                        'Pioneer MRT', 'Potong Pasir MRT', 'Promenade MRT', 'Punggol MRT', 'Queenstown MRT', 'Raffles Place MRT', 'Redhill MRT',
                        'Riviera LRT', 'Rochor MRT', 'Sembawang MRT', 'Sengkang MRT', 'Serangoon MRT', 'Simei MRT', 'Sixth Avenue MRT', 
                        'Somerset MRT', 'Springleaf MRT', 'Stadium MRT', 'Stevens MRT', 'Sumang LRT', 'Tai Seng MRT', 'Tampines MRT', 
                        'Tampines East MRT', 'Tampines West MRT', 'Tanah Merah MRT', 'Tanjong Pagar MRT', 'Tanjong Rhu MRT', 'Teck Lee LRT', 
                        'Telok Ayer MRT', 'Telok Blangah MRT', 'Thanggam LRT', 'Tiong Bahru MRT', 'Toa Payoh MRT', 
                        'Tuas Crescent MRT', 'Tuas Link MRT', 'Tuas West Road MRT', 'Ubi MRT', 'Upper Changi MRT', 
                        'Woodlands MRT', 'Woodlands South MRT', 'Woodlands North MRT', 'Yew Tee MRT', 'Yio Chu Kang MRT', 'Yishun MRT']
    # Future stations - 'Tampines North MRT', 'Tengah MRT'

    mrt_coordinates = {}

    for mrt in mrt_stations:
        try:
            response = requests.get(f"https://developers.onemap.sg/commonapi/search?searchVal={mrt}&returnGeom=Y&getAddrDetails=Y")
            response.raise_for_status()
            data = response.json()
            # string (lat,long) as key
            # mrt_coordinates[f"{data['results'][0]['LATITUDE']},{data['results'][0]['LONGITUDE']}"] = mrt
            mrt_coordinates[mrt] = (float(data['results'][0]['LATITUDE']),float(data['results'][0]['LONGITUDE']))
        except HTTPError as http_err:
            print(f'HTTP error occurred: {http_err}')
        except Exception as err:
            print(f'Other error occurred: {err}')
            print(f'Error for {mrt, data}')

    with open('static/mrt_dict.json', 'w')as f:
        json.dump(mrt_coordinates, f, indent=4)

def get_mrt_coordinates(location = 'static/mrt_dict.json'):
    with open(location, 'r') as f:
        file = f.read()
        data = json.loads(file)
        return data


In [3]:
mrt_coordinates = get_mrt_coordinates()

In [4]:
start = "1.32283703302242,103.939124525951"
routeType = ['walk', 'pt', 'drive', 'cycle']
end = "1.36126901451361,103.854642365822"

start = "1.32953680475668,103.940406562732"
end = "1.44058856161847,103.800990519771"

for num in range(2):
    try:
        if num == 0:
            response = requests.get(f"https://developers.onemap.sg/privateapi/routingsvc/route?start={start}&end={end}&routeType={routeType[num]}&token={credentials}")
        else:
            date = '01-26-2023'
            time = '07:35:00'
            mode = 'TRANSIT'
            maxWalkDistance = 1000
            numItineraries = 3
            response = requests.get(f"https://developers.onemap.sg/privateapi/routingsvc/route?start={start}&end={end}&routeType={routeType[num]}&token={credentials}&date={date}&time={time}&mode={mode}&maxWalkDistance={maxWalkDistance}&numItineraries={numItineraries}")
        response.raise_for_status()
        data = response.json()
        # pprint(data)
    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')
        print(response.json())
    except Exception as err:
        print(f'Other error occurred: {err}')
    
    if num == 0:
        # walking will only have 1 best route
        walk_time = data['route_summary']['total_time']
        walk_distance = data['route_summary']['total_distance']
        print(f'Walk time: {walk_time}')
        print(f'Walk distance: {walk_distance}')
    else:
        pprint(data)


Walk time: 18681
Walk distance: 25950
{'debugOutput': {'pathCalculationTime': 69,
                 'pathTimes': [37, 20, 12],
                 'precalculationTime': 72,
                 'renderingTime': 1,
                 'timedOut': False,
                 'totalTime': 142},
 'elevationMetadata': {'ellipsoidToGeoidDifference': 7.3773084368695,
                       'geoidElevation': False},
 'plan': {'date': 1674689700000,
          'from': {'lat': 1.3295368047567,
                   'lon': 103.94040656273,
                   'name': 'Origin',
                   'orig': '',
                   'vertexType': 'NORMAL'},
          'itineraries': [{'duration': 4074,
                           'elevationGained': 0,
                           'elevationLost': 0,
                           'endTime': 1674694039000,
                           'fare': '1.65',
                           'legs': [{'agencyTimeZoneOffset': 28800000,
                                     'arrivalDelay': 0,
        